## **1) Importing Python Packages for GAN**


In [1]:
from keras.datasets import cifar10, mnist
from keras.models import Sequential
from keras.layers import Reshape
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import Dropout
from keras.layers.advanced_activations import LeakyReLU
from tensorflow.keras.optimizers import Adam
import numpy as np
import tensorflow as tf
!mkdir generated_images

## **2) Parameters for Neural Networks & Data**

In [2]:
img_width = 32
img_height = 32
channels = 3
img_shape = (img_width, img_height, channels)
latent_dim = 100
adam = Adam(lr=0.0002)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


## **3) Building Generator**





In [3]:
#we wish to have an output shape of (32,32,3) upscaling the initial 256*4*4 using Conv2DTranspose

def build_generator():
  model = Sequential([
    Dense(256 * 4 * 4, input_dim = latent_dim),
    LeakyReLU(alpha = 0.2),
    Reshape((4,4,256)),

    #upscale an image
    Conv2DTranspose(128, (4,4), strides = (2,2), padding = "same"),
    LeakyReLU(alpha = 0.2),

    Conv2DTranspose(128, (4,4), strides = (2,2), padding = "same"),
    LeakyReLU(alpha = 0.2),

    Conv2DTranspose(128, (4,4), strides = (2,2), padding = "same"),
    LeakyReLU(alpha = 0.2),

    Conv2D(3,(3,3),activation = "tanh", padding = "same")
  ])

  return model

generator = build_generator()

## **4) Building Discriminator**

In [4]:
def build_discriminator():

  model = Sequential([
    #input match the input data
    Conv2D(64, (3,3), padding = "same", input_shape = img_shape),
    LeakyReLU(alpha = 0.2),

    Conv2D(128, (3,3), padding = "same", input_shape = img_shape),
    LeakyReLU(alpha = 0.2),

    Conv2D(128, (3,3), padding = "same", input_shape = img_shape),
    LeakyReLU(alpha = 0.2),

    Conv2D(256, (3,3), padding = "same", input_shape = img_shape),
    LeakyReLU(alpha = 0.2),

    #flatten alllows us to have a binary output
    Flatten(),
    Dropout(0.4),

    #1 outpu to classify fake from real
    Dense(1, activation = "sigmoid")
    
  ])
  
  model.summary()
  return model

discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 32, 32, 64)        1792      
                                                                 
 leaky_re_lu_4 (LeakyReLU)   (None, 32, 32, 64)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 32, 32, 128)       73856     
                                                                 
 leaky_re_lu_5 (LeakyReLU)   (None, 32, 32, 128)       0         
                                                                 
 conv2d_3 (Conv2D)           (None, 32, 32, 128)       147584    
                                                                 
 leaky_re_lu_6 (LeakyReLU)   (None, 32, 32, 128)       0         
                                                                 
 conv2d_4 (Conv2D)           (None, 32, 32, 256)      

## **5) Connecting Neural Networks to build GAN**

In [5]:
GAN = Sequential()
GAN.add(generator)
GAN.add(discriminator)

GAN.layers[1].trainable = False
GAN.compile(loss='binary_crossentropy', optimizer=adam)

In [6]:
GAN.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 32, 32, 3)         1466115   
                                                                 
 sequential_1 (Sequential)   (None, 1)                 780545    
                                                                 
Total params: 2,246,660
Trainable params: 1,466,115
Non-trainable params: 780,545
_________________________________________________________________


## **6) Outputting Images**


In [7]:
#@title
## **7) Outputting Images**
import matplotlib.pyplot as plt
import glob
import imageio
import PIL

save_name = 0.00000000

def save_imgs(epoch):
    r, c = 2, 2
    noise = np.random.normal(0, 1, (r * c, latent_dim))
    gen_imgs = generator.predict(noise)
    global save_name
    save_name += 0.00000001
    # print("%.8f" % save_name)

    # Rescale images 0 - 1
    # gen_imgs = 0.5 * gen_imgs + 0.5
    gen_imgs = (gen_imgs + 1) / 2.0
    # gen_imgs = gen_imgs * 255

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt])
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("generated_images/%.8f.png" % save_name)
    plt.close()

## **7) Training GAN**

In [ ]:
def train(epochs, batch_size=64, save_interval=200):
  (X_train, _), (_, _) = cifar10.load_data()

  #Rescale data between -1 and 1
  X_train = X_train / 127.5 -1.
  bat_per_epo = int(X_train.shape[0] / batch_size)
  
  #adds a dimension at th end, for the channel for gray pictures from (28,28) to (28,28,1), but for RGB we already have (28,28,3) so no neeed
  #X_train = np.expand_dims(X_train, axis=3)

  #Create our Y for our Neural Networks
  valid = np.ones((batch_size, 1))
  fakes = np.zeros((batch_size, 1))

  for epoch in range(epochs):
    for j in range(bat_per_epo):
      #Get Random Batch
      idx = np.random.randint(0, X_train.shape[0], batch_size)
      imgs = X_train[idx]

      #Generate Fake Images
      noise = np.random.normal(0, 1, (batch_size, latent_dim))
      gen_imgs = generator.predict(noise)

      #Train discriminator
      d_loss_real = discriminator.train_on_batch(imgs, valid)
      d_loss_fake = discriminator.train_on_batch(gen_imgs, fakes)
      d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

      noise = np.random.normal(0, 1, (batch_size, latent_dim))
      
      #inverse y label
      g_loss = GAN.train_on_batch(noise, valid)

      print("******* %d %d [D loss: %f, acc: %.2f%%] [G loss: %f]" % (epoch, j, d_loss[0], 100* d_loss[1], g_loss))

    save_imgs(epoch)


train(30000, batch_size=64, save_interval=200)

170508288/170498071 [==============================] - 6s 0us/step
******* 0 0 [D loss: 0.691988, acc: 39.84%] [G loss: 0.684648]
******* 0 1 [D loss: 0.475138, acc: 50.00%] [G loss: 0.675276]
******* 0 2 [D loss: 0.400196, acc: 50.00%] [G loss: 0.663944]
******* 0 3 [D loss: 0.380146, acc: 50.00%] [G loss: 0.668222]
******* 0 4 [D loss: 0.372777, acc: 50.00%] [G loss: 0.703469]
******* 0 5 [D loss: 0.340337, acc: 98.44%] [G loss: 0.792401]
******* 0 6 [D loss: 0.284709, acc: 100.00%] [G loss: 0.972625]
******* 0 7 [D loss: 0.206501, acc: 100.00%] [G loss: 1.292089]
******* 0 8 [D loss: 0.145881, acc: 100.00%] [G loss: 1.718528]
******* 0 9 [D loss: 0.103858, acc: 100.00%] [G loss: 2.118136]
******* 0 10 [D loss: 0.223651, acc: 96.88%] [G loss: 1.966610]
******* 0 11 [D loss: 0.204999, acc: 97.66%] [G loss: 1.678829]
******* 0 12 [D loss: 0.150400, acc: 100.00%] [G loss: 1.535702]
******* 0 13 [D loss: 0.187544, acc: 100.00%] [G loss: 1.506113]
******* 0 14 [D loss: 0.220925, acc: 99.2

In [ ]:
noise = np.random.normal(0, 1, (1,latent_dim))
gen_imgs = generator.predict(noise)

In [ ]:
gen_imgs = (gen_imgs + 1) / 2.0
plt.imshow(gen_imgs)

### **8) Making GIF**

In [ ]:
# Display a single image using the epoch number
# def display_image(epoch_no):
#   return PIL.Image.open('generated_images/%.8f.png'.format(epoch_no))

anim_file = 'dcgan.gif'

with imageio.get_writer(anim_file, mode='I') as writer:
  filenames = glob.glob('generated_images/*.png')
  filenames = sorted(filenames)
  for filename in filenames:
    image = imageio.imread(filename)
    writer.append_data(image)
  image = imageio.imread(filename)
  writer.append_data(image)